In [2]:
# Import dependencies
import pandas as pd
import os
# import csv

## Import The Data

In [3]:
# Specify Filepaths
locations_inpath = "Resources/CitiesWGeolocation.csv"
cost_analysis_inpath = "Resources/cost_analysis.csv"
weather_inpath = "Resources/weather.csv"
yelp_inpath = "Resources/final_activities.csv"
chat_inpath = "Resources/chatgpt_activities.csv"
# activity_inpath = "Resources/activity_list.csv"

In [5]:
# Import Files
locations_raw_df = pd.read_csv(locations_inpath,index_col=[0])
cost_analysis_df = pd.read_csv(cost_analysis_inpath,index_col=[0])
weather_df = pd.read_csv(weather_inpath,index_col=[0])
yelp_df = pd.read_csv(yelp_inpath)
chat_df = pd.read_csv(chat_inpath,index_col=[0])
# activity_df = pd.read_csv(activity_inpath)

### Clean Datasets Before Merging

In [6]:
locations_raw_df.info()
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('Kingdom of the Netherlands','Netherlands')
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('Sr Lanka','Sri Lanka')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 0 to 236
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            220 non-null    object 
 1   State/Province  75 non-null     object 
 2   Country         220 non-null    object 
 3   Latitude        220 non-null    float64
 4   Longitude       220 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [7]:
cost_analysis_df.sort_values(by="total_rank")

,city,State/Province,country,Latitude,Longitude,meal,ticket,taxi,meal_ranking,ticket_ranking,taxi_ranking,total_rank
219,Hoi An,NaN,Vietnam,15.879814,108.327394,19.252941,0.313750,0.981053,1,1.0,1.0,1.0
50,Beijing,NaN,China,39.906217,116.391276,30.730000,0.700000,2.380000,1,1.0,1.0,1.0
52,Medellin,NaN,Colombia,6.244338,-75.573553,18.340000,0.520000,2.350000,1,1.0,1.0,1.0
83,Jaipur,Rajasthan,India,26.915458,75.818982,9.790000,0.310000,0.850000,1,1.0,1.0,1.0
84,Agra,Uttar Pradesh,India,27.175255,78.009816,7.350000,NaN,0.710000,1,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
160,Geneva,NaN,Switzerland,46.201756,6.146601,126.810000,3.170000,9.510000,5,5.0,5.0,5.0
161,St. Moritz,Switzerland,Switzerland,46.497896,9.839243,109.264737,3.925143,11.284044,5,5.0,5.0,5.0
82,Reykjavik,NaN,Iceland,64.145981,-21.942237,105.920000,3.460000,7.780000,5,5.0,5.0,5.0
123,Oslo,NaN,Norway,59.913330,10.738970,108.580000,3.920000,11.920000,5,5.0,5.0,5.0


In [8]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2640 entries, 0 to 2639
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Latitude               2640 non-null   float64
 1   Longitude              2640 non-null   float64
 2   Month                  2640 non-null   int64  
 3   Average Temperature    2640 non-null   float64
 4   Average No Sunny Days  2640 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 123.8 KB


In [9]:
yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   162 non-null    float64
 1   longitude  162 non-null    float64
 2   act1       162 non-null    object 
 3   act2       147 non-null    object 
 4   act3       144 non-null    object 
 5   act4       142 non-null    object 
 6   act5       141 non-null    object 
dtypes: float64(2), object(5)
memory usage: 9.0+ KB


In [10]:
df_list = [locations_raw_df,cost_analysis_df,weather_df,chat_df,yelp_df]

for i in range(len(df_list)): 
    df_list[i].columns = df_list[i].columns.str.lower()
    for x in range(len(df_list[i].columns)):
        df = df_list[i]
        col = df.columns[x]
        try:
            df[col] = df[col].str.title()
            df[col] = df[col].str.strip()
            df[col] = df[col].astype(str)
        except:
            df[col] = df[col].round()
            pass


cost_analysis_df = cost_analysis_df.rename(columns={"total_rank":"cost_rank"})
cost_analysis_df = cost_analysis_df[["city","state/province","country","latitude","longitude","cost_rank"]]
weather_df = weather_df.rename(columns={"average temperature":"avg_temp","average no sunny days":"avg_num_sun_days"})

## Merge All Dataframes Together

In [11]:
# Merge Locations onto Cost Analysis (step1_df)
step1_df = pd.merge(locations_raw_df,cost_analysis_df,how='inner',on=["city","country","latitude","longitude","state/province"])
step1_df = step1_df.drop_duplicates(subset=["city","country","latitude","longitude"])
step1_df.sort_values(by=["cost_rank"])

,city,state/province,country,latitude,longitude,cost_rank
109,Siem Reap,nan,Cambodia,13.0,104.0,1.0
99,São Paulo,nan,Brazil,-24.0,-47.0,1.0
98,Jakarta,nan,Indonesia,-6.0,107.0,1.0
96,Chiang Mai,nan,Thailand,19.0,99.0,1.0
94,Bucharest,nan,Romania,44.0,26.0,1.0
...,...,...,...,...,...,...
121,Berlin,nan,Germany,53.0,13.0,5.0
126,New York,nan,United States,41.0,-74.0,5.0
171,Brussels,nan,Belgium,51.0,4.0,5.0
201,Rotterdam,nan,Netherlands,52.0,4.0,5.0


In [12]:
#before merging weather onto main_df, do a little cleaning up.
weather_df['avg_temp']=(weather_df['avg_temp']*9/5)+32 #Change to Fahrenheit
weather_df.head()

,latitude,longitude,month,avg_temp,avg_num_sun_days
0,38.0,-84.0,4,51.8,9.0
1,38.0,-84.0,5,68.0,10.0
2,38.0,-84.0,6,75.2,13.0
3,38.0,-84.0,7,75.2,13.0
4,38.0,-84.0,8,71.6,9.0


In [13]:
#create a two item temp/sun list keyed to an outer list of months in order to bring weather into one row and save for later 
weather_df= weather_df.groupby(['latitude', 'longitude']).apply(lambda x: x[['month', 'avg_temp', 'avg_num_sun_days']]
                                                              .values.tolist()).reset_index(name='weather')

# Create a dictionary for each row to store month, avg_temp, and avg_num_sun_days
weather_df['weather'] = weather_df['weather'].apply(lambda x: [{'month': m[0], 'avg_temp': m[1], 'avg_num_sun_days': m[2]} for m in x])


In [14]:
weather_df.head()

,latitude,longitude,weather
0,-45.0,169.0,"[{'month': 4.0, 'avg_temp': 48.2, 'avg_num_sun..."
1,-38.0,145.0,"[{'month': 4.0, 'avg_temp': 64.4, 'avg_num_sun..."
2,-37.0,142.0,"[{'month': 4.0, 'avg_temp': 64.4, 'avg_num_sun..."
3,-37.0,175.0,"[{'month': 4.0, 'avg_temp': 62.6, 'avg_num_sun..."
4,-35.0,-56.0,"[{'month': 4.0, 'avg_temp': 64.4, 'avg_num_sun..."


In [15]:
# Merge (step1_df) onto Weather (step2_df)  
step2_df = pd.merge(step1_df,weather_df,how='inner',on=["latitude","longitude"])
step2_df = step2_df.drop_duplicates(subset=["city","country","latitude","longitude"],keep='first')
step2_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,weather
104,Krakow,nan,Poland,50.0,20.0,1.0,"[{'month': 4.0, 'avg_temp': 42.8, 'avg_num_sun..."
36,Marrakech,nan,Morocco,32.0,-8.0,1.0,"[{'month': 4.0, 'avg_temp': 68.0, 'avg_num_sun..."
163,Pokhara,nan,Nepal,28.0,84.0,1.0,"[{'month': 4.0, 'avg_temp': 73.4, 'avg_num_sun..."
116,Colombo,nan,Sri Lanka,7.0,80.0,1.0,"[{'month': 4.0, 'avg_temp': 80.6, 'avg_num_sun..."
44,Jaipur,Rajasthan,India,27.0,76.0,1.0,"[{'month': 4.0, 'avg_temp': 91.4, 'avg_num_sun..."
...,...,...,...,...,...,...,...
68,Odense,Fyn,Denmark,55.0,10.0,5.0,"[{'month': 4.0, 'avg_temp': 44.6, 'avg_num_sun..."
176,Munich,nan,Germany,48.0,12.0,5.0,"[{'month': 4.0, 'avg_temp': 46.4, 'avg_num_sun..."
174,Stockholm,nan,Sweden,59.0,18.0,5.0,"[{'month': 4.0, 'avg_temp': 41.0, 'avg_num_sun..."
124,New York,nan,United States,41.0,-74.0,5.0,"[{'month': 4.0, 'avg_temp': 53.6, 'avg_num_sun..."


In [16]:
# Merge (step2_df) onto Chat (step3_df)
step3_df = pd.merge(step2_df,chat_df,how='left',on=['city','country'])
step3_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,weather,act1,act2,act3,act4,act5
104,Krakow,nan,Poland,50.0,20.0,1.0,"[{'month': 4.0, 'avg_temp': 42.8, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN
36,Marrakech,nan,Morocco,32.0,-8.0,1.0,"[{'month': 4.0, 'avg_temp': 68.0, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN
163,Pokhara,nan,Nepal,28.0,84.0,1.0,"[{'month': 4.0, 'avg_temp': 73.4, 'avg_num_sun...",Hiking,Tours,Hot Air Balloons,Spas,Horsebackriding
116,Colombo,nan,Sri Lanka,7.0,80.0,1.0,"[{'month': 4.0, 'avg_temp': 80.6, 'avg_num_sun...",Beaches,Tours,Spas,Snorkeling,Sailing
44,Jaipur,Rajasthan,India,27.0,76.0,1.0,"[{'month': 4.0, 'avg_temp': 91.4, 'avg_num_sun...",Tours,Hot_Air_Balloons,Horsebackriding,Spas,Hiking
...,...,...,...,...,...,...,...,...,...,...,...,...
68,Odense,Fyn,Denmark,55.0,10.0,5.0,"[{'month': 4.0, 'avg_temp': 44.6, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN
176,Munich,nan,Germany,48.0,12.0,5.0,"[{'month': 4.0, 'avg_temp': 46.4, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN
174,Stockholm,nan,Sweden,59.0,18.0,5.0,"[{'month': 4.0, 'avg_temp': 41.0, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN
124,New York,nan,United States,41.0,-74.0,5.0,"[{'month': 4.0, 'avg_temp': 53.6, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN


In [17]:
locations_raw_df["country"].nunique()

83

In [18]:
# Merge (step3_df) onto Yelp (step4_df)
step4_df=pd.merge(step3_df,yelp_df,how='inner',on=['latitude','longitude'])
step4_df.sort_values(by='cost_rank')

,city,state/province,country,latitude,longitude,cost_rank,weather,act1_x,act2_x,act3_x,act4_x,act5_x,act1_y,act2_y,act3_y,act4_y,act5_y
164,La Paz,nan,Bolivia,-16.0,-68.0,1.0,"[{'month': 4.0, 'avg_temp': 46.4, 'avg_num_sun...",Hiking,Tours,Hot Springs,Hang Gliding,Horsebackriding,Ziplining,nan,nan,nan,nan
94,Krakow,nan,Poland,50.0,20.0,1.0,"[{'month': 4.0, 'avg_temp': 42.8, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Ziplining,Beaches,Hot_Air_Balloons,Hiking,Tours
95,Kuala Lumpur,nan,Malaysia,3.0,102.0,1.0,"[{'month': 4.0, 'avg_temp': 80.6, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Ziplining,Hot_Air_Balloons,Hiking,Massage,Tours
96,Ho Chi Minh City,nan,Vietnam,11.0,107.0,1.0,"[{'month': 4.0, 'avg_temp': 84.2, 'avg_num_sun...",Tours,Spas,Massage,Hot Springs,Hiking,Ziplining,nan,nan,nan,nan
92,Rio De Janeiro,nan,Brazil,-23.0,-43.0,1.0,"[{'month': 4.0, 'avg_temp': 73.4, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Beaches,Ziplining,Tours,Hiking,Massage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Brussels,nan,Belgium,51.0,4.0,5.0,"[{'month': 4.0, 'avg_temp': 53.6, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Ziplining,Massage,Tours,Hanggliding,Beaches
143,Stockholm,nan,Sweden,59.0,18.0,5.0,"[{'month': 4.0, 'avg_temp': 41.0, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Ziplining,Tours,Massage,Beaches,Horsebackriding
142,London,nan,United Kingdom,52.0,-0.0,5.0,"[{'month': 4.0, 'avg_temp': 53.6, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Tours,Ziplining,Massage,Horsebackriding,Hot_Air_Balloons
161,Copenhagen,nan,Denmark,56.0,13.0,5.0,"[{'month': 4.0, 'avg_temp': 42.8, 'avg_num_sun...",NaN,NaN,NaN,NaN,NaN,Tours,Ziplining,Beaches,Hiking,Massage


In [19]:
step4_df['act1_x'].fillna(step4_df['act1_y'],inplace=True)
step4_df['act2_x'].fillna(step4_df['act2_y'],inplace=True)
step4_df['act3_x'].fillna(step4_df['act3_y'],inplace=True)
step4_df['act4_x'].fillna(step4_df['act4_y'],inplace=True)
step4_df['act5_x'].fillna(step4_df['act5_y'],inplace=True)
step4_df.drop(['act1_y','act2_y','act3_y','act4_y','act5_y'],axis=1,inplace=True)
step4_df.columns=step4_df.columns.str.rstrip('_x')
step4_df.head()

,city,state/province,country,latitude,longitude,cost_rank,weather,act1,act2,act3,act4,act5
0,Lexington,Kentucky,United States,38.0,-84.0,4.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...",Tours,Paddleboarding,Massage,Ziplining,Horsebackriding
1,San Diego,California,United States,33.0,-117.0,4.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...",Beaches,Hiking,Snorkeling,Parasailing,Hot_Air_Balloons
2,Park City,Utah,United States,41.0,-111.0,3.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...",Tours,Massage,Hiking,Horsebackriding,Ziplining
3,Newcastle Upon Tyne,England,United Kingdom,55.0,-2.0,4.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...",Ziplining,Beaches,Massage,Tours,Horsebackriding
4,Brisbane,Queensland,Australia,-27.0,153.0,4.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...",Ziplining,Massage,Tours,Beaches,Hiking


In [20]:
# Clean final DF Activities to be one of the following list: 
#'beaches','hanggliding','horsebackriding', 'hiking','hot_air_balloons',
#'paddleboarding','parasailing','sailing','snorkeling','ziplining', 
#'Spas','hotsprings','massage','tours'
unique_activities=[]
activities=['act1','act2','act3','act4','act5']
for act in activities:
    unique_activities.extend(step4_df[act].unique())
unique_activities=list(set(unique_activities))
print(unique_activities)


['Hang Gliding', 'Sailing', 'Hotsprings', 'Hot Springs', 'Hot Air Balloons', 'Paddleboarding', 'Snorkeling', 'Wine Tours', 'Temples', 'Whale Watching', 'Hiking', 'Horsebackriding', 'Parasailing', 'Ziplining', 'Massage', 'nan', 'Spas', 'Beaches', 'Tours', 'Hot_Air_Balloons', 'Hanggliding']


In [21]:
matching_dictionary={"Hot_Air_Balloons":"Hot Air Balloons","Hanggliding":"Hang Gliding","Whale Watching":"Tours",
                    "Wine Tours":"Tours","Temples":"Tours", "Hotsprings":"Hot Springs", "Horsebackriding":"Horseback Riding"}
step4_df=step4_df.replace(matching_dictionary)
step4_df.head()

,city,state/province,country,latitude,longitude,cost_rank,weather,act1,act2,act3,act4,act5
0,Lexington,Kentucky,United States,38.0,-84.0,4.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...",Tours,Paddleboarding,Massage,Ziplining,Horseback Riding
1,San Diego,California,United States,33.0,-117.0,4.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...",Beaches,Hiking,Snorkeling,Parasailing,Hot Air Balloons
2,Park City,Utah,United States,41.0,-111.0,3.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...",Tours,Massage,Hiking,Horseback Riding,Ziplining
3,Newcastle Upon Tyne,England,United Kingdom,55.0,-2.0,4.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...",Ziplining,Beaches,Massage,Tours,Horseback Riding
4,Brisbane,Queensland,Australia,-27.0,153.0,4.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...",Ziplining,Massage,Tours,Beaches,Hiking


## Separate Out into Tables for SQL Import

### List of all SQL Tables and Columns

#### Locations
    LocationID(pk), City, Locality, Country, Latitude, Longitude

In [22]:
#break out locations and assign the step4_df index to be our locationID
locations_df=step4_df[['city','state/province','country','latitude','longitude']]
locations_df.insert(0,'LocationID',step4_df.index)
locations_df.rename(columns={'city':'City','state/province':'Locality','country':'Country',
                             'latitude':'Latitude','longitude':'Longitude'},inplace=True)
locations_df.head()

C:\Users\rhise\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,LocationID,City,Locality,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.0,-84.0
1,1,San Diego,California,United States,33.0,-117.0
2,2,Park City,Utah,United States,41.0,-111.0
3,3,Newcastle Upon Tyne,England,United Kingdom,55.0,-2.0
4,4,Brisbane,Queensland,Australia,-27.0,153.0


#### Temperature
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

In [23]:
#Split out the weather, use latitude as well just so that it's not a single column.
temperature_df=step4_df[['latitude','weather']]
temperature_df.insert(0,'LocationID',step4_df.index)
#make each month it's own column
for i in range(12):
    temperature_df[f'month{i+1}'] = temperature_df['weather'].apply(lambda x: x[i] if len(x) > i else None)
temperature_df.head()


C:\Users\rhise\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LocationID,latitude,weather,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,38.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 68.0, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 71.6, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 55.4, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 39.2, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 39.2, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 41.0, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 44.6, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 37.4, 'avg_num_sun_..."
1,1,33.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 68.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 71.6, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 68.0, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 59.0, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 55.4, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 55.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 57.2, 'avg_num_sun_..."
2,2,41.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 42.8, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 53.6, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 46.4, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 14.0, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 12.2, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 21.2, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 15.8, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 26.6, 'avg_num_sun_..."
3,3,55.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 55.4, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 66.2, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 32.0, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 41.0, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 46.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 41.0, 'avg_num_sun_..."
4,4,-27.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 66.2, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 73.4, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 73.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 75.2, 'avg_num_sun_..."


In [24]:
#make the avg temp for the month the only value IN that column
for i in range(1,13):
    monthly_col=f'month{i}'
    temperature_df[monthly_col]=temperature_df[monthly_col].apply(lambda x:x['avg_temp'])
temperature_df.head()

,LocationID,latitude,weather,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,38.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...",51.8,68.0,75.2,75.2,71.6,69.8,55.4,39.2,39.2,41.0,44.6,37.4
1,1,33.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...",60.8,64.4,68.0,69.8,75.2,71.6,68.0,59.0,55.4,57.2,55.4,57.2
2,2,41.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...",26.6,42.8,57.2,69.8,64.4,53.6,46.4,14.0,12.2,21.2,15.8,26.6
3,3,55.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...",50.0,55.4,59.0,64.4,66.2,57.2,51.8,51.8,32.0,41.0,46.4,41.0
4,4,-27.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...",69.8,69.8,59.0,57.2,59.0,64.4,66.2,69.8,69.8,73.4,73.4,75.2


In [25]:
#drop the latitude and weather columns and rename the rest - remember the month1 column is april, etc
temperature_df.drop(['latitude','weather'], axis=1, inplace=True)
month_names = {'month1':'April',
              'month2':'May',
               'month3':'June',
               'month4':'July',
               'month5':'August',
               'month6':'September',
               'month7':'October',
               'month8':'November',
               'month9':'December',
               'month10':'January',
               'month11':'February',
               'month12':'March'
              }
temperature_df.rename(columns=month_names, inplace=True)
temperature_df.head()

,LocationID,April,May,June,July,August,September,October,November,December,January,February,March
0,0,51.8,68.0,75.2,75.2,71.6,69.8,55.4,39.2,39.2,41.0,44.6,37.4
1,1,60.8,64.4,68.0,69.8,75.2,71.6,68.0,59.0,55.4,57.2,55.4,57.2
2,2,26.6,42.8,57.2,69.8,64.4,53.6,46.4,14.0,12.2,21.2,15.8,26.6
3,3,50.0,55.4,59.0,64.4,66.2,57.2,51.8,51.8,32.0,41.0,46.4,41.0
4,4,69.8,69.8,59.0,57.2,59.0,64.4,66.2,69.8,69.8,73.4,73.4,75.2


#### SunHours
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

In [26]:
sunHours_df=step4_df[['latitude','weather']]
sunHours_df.insert(0,'LocationID',step4_df.index)
#make each month it's own column
for i in range(12):
    sunHours_df[f'month{i+1}'] = sunHours_df['weather'].apply(lambda x: x[i] if len(x) > i else None)
sunHours_df.head()

C:\Users\rhise\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,LocationID,latitude,weather,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,38.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 68.0, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 71.6, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 55.4, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 39.2, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 39.2, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 41.0, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 44.6, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 37.4, 'avg_num_sun_..."
1,1,33.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 68.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 75.2, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 71.6, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 68.0, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 59.0, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 55.4, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 55.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 57.2, 'avg_num_sun_..."
2,2,41.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 42.8, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 53.6, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 46.4, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 14.0, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 12.2, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 21.2, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 15.8, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 26.6, 'avg_num_sun_..."
3,3,55.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 55.4, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 66.2, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 51.8, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 32.0, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 41.0, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 46.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 41.0, 'avg_num_sun_..."
4,4,-27.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 5.0, 'avg_temp': 69.8, 'avg_num_sun_...","{'month': 6.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 7.0, 'avg_temp': 57.2, 'avg_num_sun_...","{'month': 8.0, 'avg_temp': 59.0, 'avg_num_sun_...","{'month': 9.0, 'avg_temp': 64.4, 'avg_num_sun_...","{'month': 10.0, 'avg_temp': 66.2, 'avg_num_sun...","{'month': 11.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 12.0, 'avg_temp': 69.8, 'avg_num_sun...","{'month': 1.0, 'avg_temp': 73.4, 'avg_num_sun_...","{'month': 2.0, 'avg_temp': 73.4, 'avg_num_sun_...","{'month': 3.0, 'avg_temp': 75.2, 'avg_num_sun_..."


In [27]:
#make the SUNHOURS for the month the only value IN that column
for i in range(1,13):
    monthly_col=f'month{i}'
    sunHours_df[monthly_col]=sunHours_df[monthly_col].apply(lambda x:x['avg_num_sun_days'])
sunHours_df.head()

,LocationID,latitude,weather,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,38.0,"[{'month': 4.0, 'avg_temp': 51.8, 'avg_num_sun...",9.0,10.0,13.0,13.0,9.0,10.0,10.0,7.0,8.0,8.0,10.0,9.0
1,1,33.0,"[{'month': 4.0, 'avg_temp': 60.8, 'avg_num_sun...",11.0,14.0,14.0,14.0,12.0,11.0,11.0,8.0,7.0,8.0,9.0,9.0
2,2,41.0,"[{'month': 4.0, 'avg_temp': 26.6, 'avg_num_sun...",9.0,11.0,14.0,11.0,10.0,9.0,11.0,9.0,6.0,6.0,9.0,8.0
3,3,55.0,"[{'month': 4.0, 'avg_temp': 50.0, 'avg_num_sun...",10.0,14.0,15.0,16.0,12.0,9.0,9.0,6.0,4.0,3.0,8.0,7.0
4,4,-27.0,"[{'month': 4.0, 'avg_temp': 69.8, 'avg_num_sun...",7.0,5.0,7.0,8.0,8.0,10.0,8.0,14.0,12.0,13.0,12.0,11.0


In [28]:
sunHours_df.drop(['latitude','weather'], axis=1, inplace=True)
sunHours_df.rename(columns=month_names, inplace=True)
sunHours_df.head()

,LocationID,April,May,June,July,August,September,October,November,December,January,February,March
0,0,9.0,10.0,13.0,13.0,9.0,10.0,10.0,7.0,8.0,8.0,10.0,9.0
1,1,11.0,14.0,14.0,14.0,12.0,11.0,11.0,8.0,7.0,8.0,9.0,9.0
2,2,9.0,11.0,14.0,11.0,10.0,9.0,11.0,9.0,6.0,6.0,9.0,8.0
3,3,10.0,14.0,15.0,16.0,12.0,9.0,9.0,6.0,4.0,3.0,8.0,7.0
4,4,7.0,5.0,7.0,8.0,8.0,10.0,8.0,14.0,12.0,13.0,12.0,11.0


#### Activities
    LocationID(pk),  Activity1, Activity2, Activity3, Activity4, Activity5

In [29]:
activities_df=step4_df[['act1','act2','act3','act4','act5']]
activities_df.insert(0,'LocationID',step4_df.index)
activities_df.rename(columns={'act1':'Activity1','act2':'Activity2','act3':'Activity3','act4':'Activity4',
                              'act5':'Activity5'},inplace=True)
activities_df.head()

C:\Users\rhise\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,LocationID,Activity1,Activity2,Activity3,Activity4,Activity5
0,0,Tours,Paddleboarding,Massage,Ziplining,Horseback Riding
1,1,Beaches,Hiking,Snorkeling,Parasailing,Hot Air Balloons
2,2,Tours,Massage,Hiking,Horseback Riding,Ziplining
3,3,Ziplining,Beaches,Massage,Tours,Horseback Riding
4,4,Ziplining,Massage,Tours,Beaches,Hiking


#### ActivityList
    ActivityID(pk), Name, Image, Attribution, Link

In [30]:
# add ActivityID
#activityList_df   #use strings as ID, activity_listDF should be good to add.

#### Cost
    Location ID, total_rank

In [31]:
cost_df = step4_df[["cost_rank"]]
cost_df.insert(0,'LocationID',step4_df.index)
cost_df

,LocationID,cost_rank
0,0,4.0
1,1,4.0
2,2,3.0
3,3,4.0
4,4,4.0
...,...,...
169,169,5.0
170,170,4.0
171,171,5.0
172,172,5.0


## Export All Data Into SQL Tables

In [32]:
# Create outpaths based on ERD
locations_outpath = "Resources/SQL_tables/locations.csv"
temperature_outpath = "Resources/SQL_tables/temperature.csv"
sunHours_outpath = "Resources/SQL_tables/sunHours.csv"
activities_outpath = "Resources/SQL_tables/activities.csv"
cost_outpath = "Resources/SQL_tables/cost.csv"
#activityList_outpath = "../Resources/SQL_tables/activityList.csv"

In [30]:
# Export all dataframes
locations_df.to_csv(locations_outpath,index=False)
temperature_df.to_csv(temperature_outpath,index=False)
sunHours_df.to_csv(sunHours_outpath,index=False)
activities_df.to_csv(activities_outpath,index=False)
cost_df.to_csv(cost_outpath,index=False)
#activityList_df.to_csv(activityList_outpath,index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/SQL_tables/locations.csv'